# <center> MSBA 6461: Advanced AI for Natural Language Processing </center>
<center> Summer 2025, Mochen Yang </center>

## <center> Attention Mechanism </center>

# Table of Contents
1. [Attention Mechanism](#attention)
    - [What is it and Why do We Need it?](#attention_motivation)
    - [Technical Details of Attention Mechanism](#attention_tech)
    - [Implement Attention Mechanism in Keras](#attention_implement)
    - [A General Formulation of Attention Mechanism](#attention_general)
1. [Additional Resources](#resource)

# Attention Mechanism <a name="attention"></a>

## What is it and Why do We Need it? <a name="attention_motivation"></a>

The basic encoder-decoder architecture discussed in the last module has achieved significant successes in applications such as machine translation. However, it also has some notable limitations. One important limitation that motivated the attention mechanism is the observation that **different parts of the input sequence are not equally important for predicting the output sequence** (see the simple example below). The basic encoder-decoder architecture cannot capture this aspect, because for each input sequence we only get a _single_ context vector that is used to generate the entire output sequence.

![An Example of Why We Need Attention](images/attention.gif)

image credit: https://medium.com/eleks-labs/neural-machine-translation-with-attention-mechanism-step-by-step-guide-989adb12127b

This also naturally gives rise to the basic idea behind the attention mechanism: Instead of a single context vector, we now compute one context vector specifically for generating one word in the output sequence. That context vector should encode the information from input sequence that is most useful for predicting the target word in output sequence. In other words, we "align" the context vector for each target word in the output sequence with words in the input sequence.

## Technical Details of Attention Mechanism <a name="attention_tech"></a>

![Illustration of Attention Mechanism](images/attention_detail.png)

image credit: Figure 1 in https://arxiv.org/pdf/1409.0473.pdf. 

<font color="blue">Note:</font> I'm going to use slightly different notations than what's in the above figure, in order to be consistent with other parts of this notebook. Specifically, I will use $t$ to index positions in the input sequence and $i$ to index positions in the output sequence (whereas they are $1, \ldots, T$ and $t$ in the figure). Accordingly, I use $h_t^{(encoder)}$ and $h_i^{(decoder)}$ to represent encoder/decoder hidden states, whereas the same things are denoted as $h_t$ and $s_t$ in the above figure.

**The encoder RNN**: same as the encoder step in a standard encoder-decoder architecture, except that we often use a bi-directional RNN (rather than a one-directional RNN). <font color="blue">Intuition for using bi-directional RNN:</font> we want the hidden states of the encoder RNN to contain information of both the preceding and following words in the input sequence, to help better learn the "alignment" with the target word. Formally, the forward and backward pass are:
$$\overrightarrow{h_t^{(encoder)}} = f(\overrightarrow{h_{t-1}^{(encoder)}}, x_t)$$
$$\overleftarrow{h_t^{(encoder)}} = f(\overleftarrow{h_{t+1}^{(encoder)}}, x_t)$$
and we concatenate the two to form the hidden state of encoder RNN at time $t$, i.e.,
$$h_t^{(encoder)} = \big[\overrightarrow{h_t^{(encoder)}}, \overleftarrow{h_t^{(encoder)}} \big]$$

**The context vector**: the context vector for target word $i$ (in the output sequence) is a **weighted sum** of all encoder hidden states:
$$\boldsymbol{C_i} = \sum_{t=1}^{T} \alpha_{it} h_t^{(encoder)}$$
Here, $\alpha_{it}$ are attention weights and, intuitively, they specify "how much attention" should be paid to each position in the input sequence when deriving the representation of $i$-th word in the output sequence. So where does the weights, $\alpha_{it}$, come from? They are learned / trained jointly with other parameters as part of the entire model. More specifically, 
$$\alpha_{it} = \frac{\exp(e_{it})}{\sum_{k=1}^T \exp(e_{ik})}$$
where $e_{it}$ are often referred to as "scores" (and sometimes "energy"). Researchers have proposed multiple types of attention mechanisms, which differ in the way $e_{it}$ are computed. For example:
- Additive Attention (aka Bahdanau Attention): $e_{it} = tanh([h_{i-1}^{(decoder)}, h_t^{(encoder)}], \boldsymbol{W})$. In other words, the scores are computed via a (standard) feed-forward neural network with a single hidden layer and $tanh$ activation.
- Dot-Product Attention (often used in transformer architecture): $e_{it} = (h_{i-1}^{(decoder)})^{'} h_t^{(encoder)}$, i.e., the dot product between $h_{i-1}^{(decoder)}$ and $h_t^{(encoder)}$.
- Other types of attention mechanisms: I highly recommend taking a look at [this article](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) for details.

**The decoder RNN**: at time step $i$ of the decoder RNN, it takes the hidden state from step $i-1$ as well as the context vector $\boldsymbol{C_i}$ as input to compute the hidden state at step $t$ and then produce a prediction at that step. So,
1. Compute next hidden state as $h_i^{(decoder)} = f(h_{i-1}^{(decoder)},y_{i-1},\boldsymbol{C_i})$. The context vector $\boldsymbol{C_i}$ is concatenated with the other inputs and feed into the activation function;
2. Predict $\hat{y_i} = softmax(h_i^{(decoder)})$ as the next word in sequence;
3. Repeat steps 1-2 until termination.

## Attention Mechanism Implementation <a name="attention_implement"></a>

The [`tf.keras.layers.Attention`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Attention) implements both the additive attention (set `score_mode = "concat"`) and the dot product attention (set `score_mode = "dot"`). For other types of attention mechanisms, you may need to do some implementation yourself.

## A General Formulation of Attention Mechanism <a name="attention_general"></a>

Attention mechanism, in general, can be formulated using the (somewhat abstract and not very intuitive) terms of "query", "key", and "value". In NLP settings:
- Query $Q$ typically refers to words in the output / target sequence;
- Key $K$ and Value $V$ typically refer to words in the input sequence.

Then, the context vector (aka context embeddings) produced by the attention mechanism is computed as:

$$Attention(Q, K, V) = \boldsymbol{\alpha} V$$

where the attention weights $\boldsymbol{\alpha}$ is a softmax transformation of certain function over $(Q, K)$ that quantifies the "alignment" between $Q$ and $K$, i.e.,

$$\boldsymbol{\alpha} = softmax(align(Q, K))$$

In additive attention, $Q$ is the decoder hidden states and $K,V$ are encoder hidden states, and $align(.)$ is the $tanh$ function applied over a concatenation of $Q$ and $K$.

In dot-product attention, $Q$ is the decoder hidden states and $K,V$ are encoder hidden states, and $align(.)$ is simply the dot product.

In general, $align(.)$ can be any function that is capable of describing the degree of "alignment", or very loosely speaking, similarity / association, between the query and key vectors.

# Additional Resources <a name="resource"></a>

- Attention Mechanism:
    - Original research paper that proposed the attention mechanism: [Neural machine translation by jointly learning to align and translate](https://arxiv.org/pdf/1409.0473.pdf?utm_source=ColumnsChannel);
    - Implementation of attention: [Neural machine translation with attention](https://www.tensorflow.org/tutorials/text/nmt_with_attention);
    - [3b1b video on attention](https://www.youtube.com/watch?v=eMlx5fFNoYc)